In [ ]:
mport pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuracoes de Estilo para os Graficos ---
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14

# --- Carregamento e Filtragem dos Dados ---
try:
    df = pd.read_csv('experiment_results.csv')
    print("Arquivo 'experiment_results.csv' carregado com sucesso!")

    # --- ETAPA DE FILTRAGEM METODOLOGICA ---
    # Mantem apenas as estrategias que sao diretamente comparaveis entre as linguas
    estrategias_para_analise = ['BaselinePreprocessor', 'RegexPreprocessor']
    df_filtrado = df[df['strategy'].isin(estrategias_para_analise)].copy()
    print(f"Dados filtrados! Analise focada nas estrategias: {estrategias_para_analise}")
    
    # Preparar dataframes separados para cada tipo de analise
    static_df = df_filtrado[df_filtrado['metric_type'] == 'static'].copy()
    dynamic_df = df_filtrado[df_filtrado['metric_type'] == 'dynamic'].dropna(subset=['generation_time']).copy()

    # --- Grafico 1: Analise do Numero de Tokens ---
    print("\nGerando Grafico 1: Comparacao do Numero Medio de Tokens...")
    token_avg = static_df.groupby(['language', 'strategy', 'model_or_tokenizer'])['num_tokens'].mean().reset_index()

    token_plot = sns.catplot(
        data=token_avg, x='strategy', y='num_tokens', hue='language',
        col='model_or_tokenizer', kind='bar', height=6, aspect=1.1,
        palette='viridis', legend_out=False
    )
    token_plot.fig.suptitle('Comparacao do Numero de Tokens (Baseline vs. Regex)', y=1.03)
    token_plot.set_axis_labels("Estrategia de Pre-Processamento", "Numero Medio de Tokens")
    token_plot.set_titles("Tokenizador: {col_name}")
    token_plot.add_legend(title='Lingua')
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Grafico 2: Analise da Fertilidade ---
    print("Gerando Grafico 2: Comparacao da Fertilidade Media...")
    fertility_avg = static_df.groupby(['language', 'strategy', 'model_or_tokenizer'])['fertility'].mean().reset_index()

    fertility_plot = sns.catplot(
        data=fertility_avg, x='strategy', y='fertility', hue='language',
        col='model_or_tokenizer', kind='bar', height=6, aspect=1.1,
        palette='plasma', legend_out=False
    )
    fertility_plot.fig.suptitle('Comparacao da Fertilidade Media (Baseline vs. Regex)', y=1.03)
    fertility_plot.set_axis_labels("Estrategia de Pre-Processamento", "Fertilidade Media (Tokens/Palavra)")
    fertility_plot.set_titles("Tokenizador: {col_name}")
    for ax in fertility_plot.axes.flat:
        ax.axhline(1.0, ls='--', color='gray', zorder=0)
    fertility_plot.add_legend(title='Lingua')
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Grafico 3: Analise do Tempo de Geracao (Latencia) ---
    print("Gerando Grafico 3: Distribuicao do Tempo de Geracao (Latencia)...")
    plt.figure(figsize=(12, 8))
    sns.boxplot(
        data=dynamic_df, x='strategy', y='generation_time',
        hue='language', palette='coolwarm'
    )
    plt.title('Distribuicao do Tempo de Geracao (Baseline vs. Regex)')
    plt.xlabel('Estrategia de Pre-Processamento')
    plt.ylabel('Tempo de Geracao (segundos)')
    plt.legend(title='Lingua')
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("Erro: Arquivo 'experiment_results.csv' nao foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado durante a geracao dos graficos: {e}")